In [2]:
print("starting")
import os

# Check if ffmpeg is installed, if not then install it
ffmpeg_installed = !command -v ffmpeg
if not ffmpeg_installed:
    !apt-get install ffmpeg

#Check if librosa is installed, if not then install it
try:
    import librosa
except Exception:
    !pip install librosa
    import librosa

#Check if madmom is installed, if not then install it
try:
    import madmom
except Exception:
    !pip install mido
    !pip install --upgrade --no-deps --force-reinstall --quiet 'git+https://github.com/CPJKU/madmom.git'
    import madmom

#Check if pyaudio is installed, if not then install it
try:
    import pyaudio
except Exception: 
    !pip install pyaudio
    import pyaudio

#Check if BeatNet is installed, if not then install it
try:
    from BeatNet.BeatNet import BeatNet
except Exception:
    !pip install BeatNet
    from BeatNet.BeatNet import BeatNet


#Check if runes-client is installed, if not then install it
try:
    import elixir_client
except ImportError:
    !pip install runes-client
    import runes_client

try:
    import runes_client
except ImportError:
  !pip install runes-client --upgrade


import runes_client.core as rune
from runes_client import DAWNetFilePath
import os
import shutil
import uuid
import numpy as np

placeholder_txt = "Enter the token generated by the DAWNet plugin"
DAWNET_TOKEN = "7643a824-9420-4397-94c3-91b2ae7e1b95" #@param {type:"string"}
rune_token = DAWNET_TOKEN

dn_cli_token = os.getenv('DN_CLI_TOKEN', None)
if dn_cli_token is not None and dn_cli_token != "":
  rune_token = dn_cli_token
else:
  if rune_token is None or rune_token == "" or rune_token == placeholder_txt:
      print("ERROR: The token provided is not valid.")
      exit()

def calculate_average_bpm(output):
    """
    Calculate a refined average BPM from BeatNet output by filtering outliers.

    :param output: numpy array of shape (n, 2) where each row is [time, beat_position]
    :return: Refined average BPM as a float
    """
    # Calculate time differences (beat durations)
    beat_durations = np.diff(output[:, 0])

    # Filter outliers
    Q1 = np.percentile(beat_durations, 25)
    Q3 = np.percentile(beat_durations, 75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    filtered_durations = beat_durations[(beat_durations >= lower_bound) & (beat_durations <= upper_bound)]

    # Calculate average beat duration from filtered data
    if len(filtered_durations) == 0:
        return 0  # Avoid division by zero if there are no beats left after filtering
    average_beat_duration = np.mean(filtered_durations)

    # Convert to BPM
    bpm = 60 / average_beat_duration

    # Round to nearest whole number
    rounded_bpm = round(bpm)

    return rounded_bpm

def extract_bpm(input_file):
    # Generate a unique directory name
    unique_dir_name = f"/tmp/{uuid.uuid4()}"
    os.makedirs(unique_dir_name, exist_ok=True)

    # Copy the input file to the new directory
    new_file_path = os.path.join(unique_dir_name, os.path.basename(input_file))
    shutil.copy(input_file, new_file_path)

    # Initialize the estimator
    estimator = BeatNet(1, mode='offline', inference_model='DBN', plot=[], thread=False)

    # Use the new directory in estimator.process
    Output = estimator.process(new_file_path)

    bpm = calculate_average_bpm(Output)

    return bpm


async def bpm_detection(input_file: DAWNetFilePath):
    try:
      bpm = extract_bpm(input_file)

      # after executing your custom code you send data back to the plugin like so ..
      await rune.output().add_file(input_file)
      await rune.output().add_log(f"BPM is: {bpm}")
      await rune.output().add_message(f"BPM is: {bpm}")
      await rune.output().send()

      return True
    except Exception as e:
      await rune.output().add_error(f"Error in arbitrary_method: {e}")

      return False


rune.set_token(token=rune_token)
rune.set_name("BeatNet")
rune.set_description("This is a Rune for detecting bpms in an audio file")
rune.register_method(bpm_detection)

rune.connect_to_server()

starting
  Using cached setuptools-69.2.0-py3-none-any.whl.metadata (6.3 kB)
  Using cached wheel-0.43.0-py3-none-any.whl.metadata (2.2 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 14.6 MB/s eta 0:00:00 MB/s eta 0:00:0101
Using cached setuptools-69.2.0-py3-none-any.whl (821 kB)
Using cached wheel-0.43.0-py3-none-any.whl (65 kB)
  Attempting uninstall: wheel
    Found existing installation: wheel 0.42.0
    Uninstalling wheel-0.42.0:
      Successfully uninstalled wheel-0.42.0
  Attempting uninstall: setuptools
    Found existing installation: setuptools 67.7.2
    Uninstalling setuptools-67.7.2:
      Successfully uninstalled setuptools-67.7.2
  Attempting uninstall: pip
    Found existing installation: pip 23.3.2
    Uninstalling pip-23.3.2:
      Successfully uninstalled pip-23.3.2
  Using cached PyAudio-0.2.14.tar.gz (47 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  error: su

ModuleNotFoundError: No module named 'pyaudio'